In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split, RepeatedStratifiedKFold, train_test_split
import sklearn.metrics

In [ ]:
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn+fp)

In [ ]:
METRICS = {
        "accuracy": make_scorer(accuracy_score),
        "precision": make_scorer(precision_score),
        "recall": make_scorer(recall_score),
        "f1": make_scorer(f1_score),
        "AUC": make_scorer(roc_auc_score, needs_proba=True),
        "specificity": make_scorer(specificity_score),
        "kappa":make_scorer(cohen_kappa_score)
}

In [ ]:
#d = pd.read_csv("data/03-sem-town-shortname.csv")
#d = pd.read_csv("data/03-pico-sem-town-shortname.csv")
#d = pd.read_csv("data/03-pico-sem-town-shortname.csv")
#d = pd.read_csv("data/03-idf.csv")
#d = pd.read_csv("data/03-idf-completo.csv")
d = pd.read_csv("data/03-idf-completo-fixedq.csv")
d.head()

In [ ]:
# get small sample of data
d = d.sample(frac=0.01)
d

In [ ]:
X, y = d.drop("target", axis=1), d["target"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## autosklearn
only works on ubuntu-based linux distros
- https://automl.github.io/auto-sklearn/master/

In [ ]:
pip3 install autosklearn

In [ ]:
import autosklearn.classification
cls = autosklearn.classification.AutoSklearnClassifier()
cls.fit(X_train, y_train)
predictions = cls.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

In [ ]:
scores = cross_validate(cls, X, y, cv=splitter, scoring=METRICS)
cls_scores = pd.DataFrame(scores)
pd.DataFrame(cls_scores.mean()).T

This will run for one hour and should result in an accuracy above 0.98\
spoiler: doesn't

In [ ]:
if __name__ == "__main__":
    X, y = sklearn.datasets.load_digits(return_X_y=True)
    X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=1)
    automl = autosklearn.classification.AutoSklearnClassifier()
    automl.fit(X_train, y_train)
    y_hat = automl.predict(X_test)
    print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))

## TPOT
- http://epistasislab.github.io/tpot/api/#classification

In [ ]:
from tpot import TPOTClassifier

In [ ]:
# define model evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search
model = TPOTClassifier(generations=5, population_size=50, cv=cv, scoring='f1', verbosity=2, random_state=1, n_jobs=-1)
# perform the search
model.fit(X, y)
# export the best model
model.export('tpot_best_model.py')

### optimized tpot

In [ ]:
# define model evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1234)
# define search
model = TPOTClassifier(generations=10, population_size=100, cv=cv, scoring='f1', verbosity=2, random_state=1234, n_jobs=-1)
# perform the search
model.fit(X, y)
# export the best model
model.export('long-tpot_best_model.py')